In [1]:
import nltk
from nltk.corpus import twitter_samples
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer        
from nltk.tokenize import TweetTokenizer
import string
import re
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from functions import process_tweet,build_freqs

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\Vidit\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [2]:
all_positive_tweets = twitter_samples.strings('positive_tweets.json')
all_negative_tweets = twitter_samples.strings('negative_tweets.json')

In [3]:
test_pos = all_positive_tweets[4000:]
train_pos = all_positive_tweets[:4000]
test_neg = all_negative_tweets[4000:]
train_neg = all_negative_tweets[:4000]

train_x = train_pos + train_neg 
test_x = test_pos + test_neg

In [4]:
train_y = np.append(np.ones((len(train_pos), 1)), np.zeros((len(train_neg), 1)), axis=0)
test_y = np.append(np.ones((len(test_pos), 1)), np.zeros((len(test_neg), 1)), axis=0)

In [5]:
print("train_y.shape = " + str(train_y.shape))
print("test_y.shape = " + str(test_y.shape))

train_y.shape = (8000, 1)
test_y.shape = (2000, 1)


In [6]:
freqs = build_freqs(train_x, train_y)

print("type(freqs) = " + str(type(freqs)))
print("len(freqs) = " + str(len(freqs.keys())))

type(freqs) = <class 'dict'>
len(freqs) = 11338


In [7]:
def extract_features(tweet, freqs):
    word_l = process_tweet(tweet)
    x = np.zeros((1, 2)) 
#     x[0,0] = 1 
    for word in word_l:
        x[0,0] += freqs.get((word, 1.0),0)
        x[0,1] += freqs.get((word, 0.0),0)
    return x

In [8]:
data = []
for i in range(len(train_x)):
    data.append(extract_features(train_x[i],freqs))

In [9]:
dataset = np.array(data)
dataset = dataset.reshape(8000,2)

In [10]:
x_train = pd.DataFrame(dataset,columns=['Positive','Negative'])

In [11]:
x_train.head()

,Positive,Negative
0,3020.0,61.0
1,3573.0,444.0
2,3005.0,115.0
3,2862.0,4.0
4,3119.0,225.0


In [12]:
from sklearn.linear_model import LogisticRegression
lr = LogisticRegression()
lr.fit(x_train,train_y)

C:\ProgramData\Anaconda3\lib\site-packages\sklearn\utils\validation.py:63: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(*args, **kwargs)


LogisticRegression()

In [13]:
def predict(tweet,freqs):
    return (lr.predict(extract_features(tweet,freqs))[0])

In [14]:
pred = []
for tweet in test_x:
    pred.append(predict(tweet,freqs))

In [15]:
from sklearn.metrics import confusion_matrix,classification_report,accuracy_score

In [16]:
confusion_matrix(test_y,pred)

array([[988,  12],
       [  5, 995]], dtype=int64)

In [17]:
print(str(accuracy_score(test_y,pred)*100)+"%")

99.15%


In [18]:
print(classification_report(test_y,pred))

              precision    recall  f1-score   support

         0.0       0.99      0.99      0.99      1000
         1.0       0.99      0.99      0.99      1000

    accuracy                           0.99      2000
   macro avg       0.99      0.99      0.99      2000
weighted avg       0.99      0.99      0.99      2000



In [19]:
lr.coef_

array([[ 0.00903432, -0.01027023]])

In [20]:
lr.intercept_

array([0.99980174])